<a href="https://colab.research.google.com/github/cappelchi/calcio_notebooks/blob/main/catboost/football_live_score_prediction_catboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[CatBoost - An In-Depth Guide [Python API]](https://coderzcolumn.com/tutorials/machine-learning/catboost-an-in-depth-guide-python#9)<br>
[Catboost](https://catboost.ai/en/docs/concepts/python-reference_pool)<br>
[Cross-Validation Techniques](https://medium.com/geekculture/cross-validation-techniques-33d389897878)

### Project config

In [1]:
try:
    import neptune.new as neptune
except:
    !pip install neptune-client >> None
    import neptune.new as neptune
#from neptune.new.integrations.tensorflow_keras import NeptuneCallback
def get_credential(frmwork = 'neptune_team'):
    with open('credential.txt', 'r') as container:
        for line in container:
            if frmwork in line:
                login, psw = line.split(' ')[1], line.split(' ')[2].split('\n')[0]
                return login, psw

ERROR: Operation cancelled by user


ModuleNotFoundError: ignored

In [ ]:
#@title Set API key for neptune.ai
set_api = True #@param {type:"boolean"}
if set_api:
    username, api_key = get_credential()

### Installations

In [ ]:
!pip install catboost >> None
!pip install deap >> None

### Downloads

In [ ]:
data_version = 'football_live_npz_230105/'
project = neptune.init_project(
    name="scomesse/football", 
    api_token = api_key
    )
project[data_version + 'dataset'].download('./dataset.npz')
params = project[data_version + 'params'].fetch()
project.stop()

### Imports

In [ ]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100
print(pd.__version__)
print(np.__version__)

In [ ]:
from tqdm import tqdm
import plotly.express as px
from scipy.stats import poisson

In [ ]:
from catboost import CatBoost
from catboost import utils
from catboost import CatBoostClassifier, CatBoostRegressor
from catboost import Pool, cv
from catboost.utils import eval_metric
np.random.seed(147)

In [ ]:
import operator
import random
from deap import base
from deap import creator
from deap import tools
from deap import algorithms

### Code

#####Functions

In [ ]:
def eaSimpleWithElitism(population, toolbox, cxpb, mutpb, ngen, stats=None,
             halloffame=None, verbose=__debug__):
    """This algorithm is similar to DEAP eaSimple() algorithm, with the modification that
    halloffame is used to implement an elitism mechanism. The individuals contained in the
    halloffame are directly injected into the next generation and are not subject to the
    genetic operators of selection, crossover and mutation.
    """
    logbook = tools.Logbook()
    logbook.header = ['gen', 'nevals'] + (stats.fields if stats else [])

    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    if halloffame is None:
        raise ValueError("halloffame parameter must not be empty!")

    halloffame.update(population)
    hof_size = len(halloffame.items) if halloffame.items else 0

    record = stats.compile(population) if stats else {}
    logbook.record(gen=0, nevals=len(invalid_ind), **record)
    if verbose:
        print(logbook.stream)

    # Begin the generational process
    for gen in range(1, ngen + 1):

        # Select the next generation individuals
        offspring = toolbox.select(population, len(population) - hof_size)

        # Vary the pool of individuals
        offspring = algorithms.varAnd(offspring, toolbox, cxpb, mutpb)

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # add the best back to population:
        offspring.extend(halloffame.items)

        # Update the hall of fame with the generated individuals
        halloffame.update(offspring)

        # Replace the current population by the offspring
        population[:] = offspring

        # Append the current generation statistics to the logbook
        record = stats.compile(population) if stats else {}
        logbook.record(gen=gen, nevals=len(invalid_ind), **record)
        if verbose:
            print(logbook.stream)

    return population, logbook

#### Prepare Data

1. регрессия
2. мультиклассовая класификация {AWAY:0,DRAW:1, HOME:2} 
3. бинарная классификация: <br>
    a. HOME vs (DRAW & AWAY)<br>
    б. DRAW vs (HOME & AWAY)<br>
    в. AWAY vs (HOME & DRAW)<br>

In [ ]:
dataset_name = './dataset.npz'
data_npz = np.load(dataset_name)
X_train, X_test = data_npz['X_train'], data_npz['X_test']

In [ ]:
#@title Выбор таргета
target_type = "regression1" #@param ["regression1", "regression2", "multiclass", "binary_home", "binary_draw", "binary_away"]
if target_type == 'regression1':
    y_train, y_test = data_npz['y_train_regression1'], data_npz['y_test_regression1']
elif target_type == 'regression2':
    y_train, y_test = data_npz['y_train_regression2'], data_npz['y_test_regression2']
elif target_type == 'multiclass':
    y_train, y_test =  data_npz['y_train_multi'], data_npz['y_test_multi']
elif target_type == 'binary_home':
    y_train, y_test =  1 * (data_npz['y_train_multi'] == 2), 1 * (data_npz['y_test_multi'] == 2)
elif target_type == 'binary_draw':
    y_train, y_test =  1 * (data_npz['y_train_multi'] == 1), 1 * (data_npz['y_test_multi'] == 1)
elif target_type == 'binary_away':
    y_train, y_test =  1 * (data_npz['y_train_multi'] == 0), 1 * (data_npz['y_test_multi'] == 0)


In [ ]:
X_train.shape, X_test.shape

In [ ]:
cols = [element for element in
params['features'].replace('[', '').replace(']','').replace(' ','').replace("'","").split(',')]

In [ ]:
train_data = Pool(X_train, y_train)
test_data = Pool(X_test, y_test)

In [ ]:
gpu_cnt = utils.get_gpu_device_count()
print("Number of GPU Count : ",gpu_cnt)

#### GA

In [ ]:
def evaluate_score(
    individual, train_data = train_data, test_data = test_data, 
    target_type = target_type, gpu_cnt = gpu_cnt
                    ):
    lr = individual[0]
    l2 = 0.1 + individual[1]
    mdepth = int(individual[2])
    minleaf = int(individual[3])
    #maxleaves = int(20 + individual[4] * 44)
    #leafmethod = ['Newton', 'Gradient', 'Exact'][int(individual[4] * 2.9)]
    params = {
        'learning_rate':lr,
        'l2_leaf_reg':l2,
        'max_depth':mdepth,
        'min_data_in_leaf':minleaf,
        #'leaf_estimation_method':leafmethod,
        'early_stopping_rounds':15,
        'verbose':0,
        'iterations':50        
    }
    if gpu_cnt > 0:
        params.update({
            'task_type':"GPU", 
            'devices':'0'
        })
    if target_type[:-1] == 'regression':
        params.update({'loss_function':'RMSE'})
        booster = CatBoostRegressor(
                        **params        
                        )
    else:
        booster = CatBoostClassifier(
                        **params        
                        )

    booster.fit(train_data, eval_set = test_data)
    #print(booster.get_best_score()['validation']['RMSE'])
    if target_type[:-1] == 'regression':
        #print(booster.get_best_score()['validation']['RMSE'], lr, l2, mdepth, minleaf)
        return booster.get_best_score()['validation']['RMSE'],
    elif target_type == 'multiclass':
        return booster.get_best_score()['validation']['MultiClass'],
    elif target_type == 'binary_home':
        return booster.get_best_score()['validation']['Logloss'],
    elif target_type == 'binary_draw':
        return booster.get_best_score()['validation']['Logloss'],
    elif target_type == 'binary_away':
        return booster.get_best_score()['validation']['Logloss'],

In [ ]:
hyper_parameters = {
    'lr':{'rand':random.uniform, 'b_low':0.01, 'b_high':0.95},
    'l2':{'rand':random.uniform,'b_low':0.01, 'b_high':0.95},
    'max_depth':{'rand':random.randint,'b_low':1, 'b_high':15},
    'min_leaf':{'rand':random.randint,'b_low':1, 'b_high':16}
            }

NUM_OF_PARAMS = len(hyper_parameters)
CROWDING_FACTOR = 20.0  # crowding factor for crossover and mutation

creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
# define the hyperparameter attributes individually:
for name, param in hyper_parameters.items():
    # "hyperparameter_0", "hyperparameter_1", ...
    print(name, param, param['b_low'], param['b_high'])
    toolbox.register(name,
                     random.uniform,
                     param['b_low'],
                     param['b_high'])
# create a tuple containing an attribute generator for each param searched:
hyperparameters = ()
for name, _ in hyper_parameters.items():
    print(name)
    hyperparameters = hyperparameters + \
                      (toolbox.__getattribute__(name),)

# create the individual operator to fill up an Individual instance:
toolbox.register("individualCreator",
                 tools.initCycle,
                 creator.Individual,
                 hyperparameters,
                 n=1)

# create the population operator to generate a list of individuals:
toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)

toolbox.register("mate",
                 tools.cxSimulatedBinaryBounded,
                 low = [value['b_low'] for key, value in hyper_parameters.items()],
                 up = [value['b_high'] for key, value in hyper_parameters.items()],
                 eta = CROWDING_FACTOR)
toolbox.register("mutate",
                 tools.mutPolynomialBounded,
                 low = [value['b_low'] for key, value in hyper_parameters.items()],
                 up = [value['b_high'] for key, value in hyper_parameters.items()],
                 eta = CROWDING_FACTOR,
                 indpb = 2.0 / NUM_OF_PARAMS)
toolbox.register('evaluate', evaluate_score) # регистрируем целевую функию
toolbox.register('select', tools.selTournament, tournsize = 2) # регистрируем схему отбора

fit_stats = tools.Statistics(key=operator.attrgetter("fitness.values"))
fit_stats.register("avg", np.mean)
fit_stats.register("std", np.std)
fit_stats.register("min", np.min)
fit_stats.register("max", np.max)

In [ ]:
# Genetic Algorithm constants:
POPULATION_SIZE = 50
P_CROSSOVER = 0.9  # probability for crossover
P_MUTATION = 0.5   # probability for mutating an individual
MAX_GENERATIONS = 10
HALL_OF_FAME_SIZE = 4

In [ ]:
elitism = True
if elitism:
    pop = toolbox.populationCreator(n = POPULATION_SIZE) # количество популяций (прогонов) на эпоху(поколение)
    hof = tools.HallOfFame(HALL_OF_FAME_SIZE)
    pop, log = eaSimpleWithElitism(
        pop, toolbox, stats=fit_stats, halloffame=hof, verbose=True,
        cxpb = P_CROSSOVER, # скрещивание
        mutpb = P_MUTATION,  # мутации
        ngen = MAX_GENERATIONS, # количество эпох - поколений 
    )

In [ ]:
if not elitism:
    pop = toolbox.populationCreator(n = POPULATION_SIZE) # количество популяций (прогонов) на эпоху(поколение)
    hof = tools.HallOfFame(HALL_OF_FAME_SIZE)
    pop, log = algorithms.eaSimple(
        pop, toolbox, stats=fit_stats, halloffame=hof, verbose=True,
        cxpb = P_CROSSOVER, # скрещивание
        mutpb = P_MUTATION,  # мутации
        ngen = MAX_GENERATIONS # количество эпох - поколений
                        )

In [ ]:
px.line(log.select('min'))

In [ ]:
tools.selBest(pop, k=4)

[[0.1739599025083567,
  0.4978084699779165,
  0.8263521520216448,
  0.955552191452285],
 [0.1739599025083567,
  0.4978084699779165,
  0.8263521520216448,
  0.955552191452285],
 [0.1739599025083567,
  0.4978084699779165,
  0.8263521520216448,
  0.955552191452285],
 [0.1739599025083567,
  0.4978084699779165,
  0.8263521520216448,
  0.7963850458524759]]

In [ ]:
best_individual = tools.selBest(pop, k=4)

In [ ]:
best_individual = [[0.1739599025083567,
  0.4978084699779165,
  0.8263521520216448,
  0.955552191452285]]

In [ ]:
num = 0
lr = best_individual[num][0]
l2 = best_individual[num][1]
mdepth = int(best_individual[num][2])
minleaf = int(best_individual[num][3])
params = {
    'learning_rate':lr,
    'l2_leaf_reg':l2,
    'max_depth':mdepth,
    'min_data_in_leaf':minleaf,
    #'leaf_estimation_method':leafmethod,
    'early_stopping_rounds':25,
    'verbose':1,
    'iterations':200
}
if gpu_cnt > 0:
    params.update({
        'task_type':"GPU", 
        'devices':'0'
    })
if target_type[:-1] == 'regression':
    params.update({'loss_function':'RMSE'})
    booster = CatBoostRegressor(
                    **params        
                    )
else:
    booster = CatBoostClassifier(
                    **params        
                    )
print(params)
booster.fit(train_data, eval_set = test_data)

{'learning_rate': 0.16656391225752104, 'l2_leaf_reg': 0.2991233879911666, 'max_depth': 9, 'min_data_in_leaf': 15, 'early_stopping_rounds': 25, 'verbose': 10, 'iterations': 200, 'loss_function': 'RMSE'}
0:	learn: 0.0597642	test: 0.0600099	best: 0.0600099 (0)	total: 3.88s	remaining: 12m 52s
10:	learn: 0.0555031	test: 0.0557724	best: 0.0557724 (10)	total: 33s	remaining: 9m 27s
20:	learn: 0.0551062	test: 0.0554163	best: 0.0554163 (20)	total: 1m 2s	remaining: 8m 55s
30:	learn: 0.0549587	test: 0.0553253	best: 0.0553253 (30)	total: 1m 26s	remaining: 7m 49s
40:	learn: 0.0548539	test: 0.0552724	best: 0.0552724 (40)	total: 1m 52s	remaining: 7m 14s
50:	learn: 0.0547722	test: 0.0552358	best: 0.0552358 (50)	total: 2m 18s	remaining: 6m 44s
60:	learn: 0.0547067	test: 0.0552192	best: 0.0552191 (58)	total: 2m 44s	remaining: 6m 13s
70:	learn: 0.0546485	test: 0.0552042	best: 0.0552042 (70)	total: 3m 12s	remaining: 5m 49s
80:	learn: 0.0545941	test: 0.0551948	best: 0.0551945 (79)	total: 3m 39s	remaining: 5

In [ ]:
#-------------multiclass -------------
if target_type == 'multiclass':
    test_preds = booster.predict(X_test, prediction_type="Class").flatten()
    train_preds = booster.predict(X_train, prediction_type="Class").flatten()
#-------------regression & binary classification
else:
    test_preds = booster.predict(X_test)
    train_preds = booster.predict(X_train)    
#test_preds[:5], train_preds[:5]

(array([1.31122664, 1.27532263, 1.24586766, 1.28241983, 1.26379012]),
 array([0.74926088, 0.7314404 , 0.67577904, 0.66547815, 0.7038123 ]))

In [ ]:
#-------------multiclass classsification-------------
if target_type == 'multiclass':
    params['train_accuracy_home'] = np.sum((y_train == 2) & (train_preds == 2)) / np.sum(train_preds == 2)
    params['train_accuracy_draw'] = np.sum((y_train == 1) & (train_preds == 1)) / np.sum(train_preds == 1)
    params['train_accuracy_away'] = np.sum((y_train == 0) & (train_preds == 0)) / np.sum(train_preds == 0)

    print("Train Accuracy Home: %.4f"%params['train_accuracy_home'])
    print("Train Accuracy Draw: %.4f"%params['train_accuracy_draw'])
    print("Train Accuracy Away: %.4f"%params['train_accuracy_away'])

    params['test_accuracy_home'] = np.sum((y_test == 2) & (test_preds == 2)) / np.sum(test_preds == 2)
    params['test_accuracy_draw'] = np.sum((y_test == 1) & (test_preds == 1)) / np.sum(test_preds == 1)
    params['test_accuracy_away'] = np.sum((y_test == 0) & (test_preds == 0)) / np.sum(test_preds == 0)

    print("Test Accuracy Home: %.4f"%params['test_accuracy_home'])
    print("Test Accuracy Draw: %.4f"%params['test_accuracy_draw'])
    print("Test Accuracy Away: %.4f"%params['test_accuracy_away'])
    #print("Test  Accuracy : %.4f"%params['train_accuracy'])

Test  RMSE : 0.0552
Train RMSE :  0.054470
Test  R2 : 0.1984
Train R2 :  0.212063


In [ ]:
if target_type == 'multiclass':
    z_train = np.zeros((3,3))
    z_test = np.zeros((3,3))
    for pred_class in range(3):
        for true_class in range(3):
            z_train[true_class, pred_class] = np.sum((train_preds == pred_class) & (y_train == true_class))
            z_test[true_class, pred_class] = np.sum((test_preds == pred_class) & (y_test == true_class))

In [ ]:
if target_type == 'multiclass':
    title_text = 'confusion matrix train'

    x = ['Away', 'Draw', 'Home']
    y = ['Away', 'Draw', 'Home']

    cm_train = px.imshow(z_train, x=x, y=y, color_continuous_scale='Purples', text_auto=True)
    cm_train.update_xaxes(title_text = 'Predicted Label')
    cm_train.update_yaxes(title_text = 'True Label')
    cm_train.update_layout(
        height = 400,
        title_text = title_text,
        title_font_size=20,
        title_x=0.5,
        paper_bgcolor='rgb(229, 237, 247)',
        plot_bgcolor='rgb(229, 237, 247)',    
        )
    cm_train.update_coloraxes(showscale=False)
    cm_train.show()

In [ ]:
if target_type == 'multiclass':
    title_text = 'confusion matrix test'

    x = ['Away', 'Draw', 'Home']
    y = ['Away', 'Draw', 'Home']

    cm_test = px.imshow(z_test, x=x, y=y, color_continuous_scale='Purples', text_auto=True)
    cm_test.update_xaxes(title_text = 'Predicted Label')
    cm_test.update_yaxes(title_text = 'True Label')
    cm_test.update_layout(
        height = 400,
        title_text = title_text,
        title_font_size=20,
        title_x=0.5,
        paper_bgcolor='rgb(229, 237, 247)',
        plot_bgcolor='rgb(229, 237, 247)',    
        )
    cm_test.update_coloraxes(showscale=False)
    cm_test.show()

In [ ]:
#-------------binary classsification-------------
if 'binary' in target_type:
    params['train_accuracy'] = booster.score(train_data)
    params['test_accuracy'] = booster.score(test_data)
    print("Test  Accuracy : %.4f"%params['train_accuracy'])
    print("Train Accuracy : %.4f"%params['test_accuracy'])

In [ ]:
#-------------binary classsification-------------
if 'binary' in target_type:
    tp = target_type.split('_')[1]
    type = tp
    params[f'train_accuracy_{type}'] = np.sum((y_train == 1) & (train_preds == 1)) / np.sum(train_preds == 1)
    params['train_accuracy_rest'] = np.sum((y_train == 0) & (train_preds == 0)) / np.sum(train_preds == 0)

    print(f"Train Accuracy {type}: %.4f"%params[f'train_accuracy_{type}'])
    print("Train Accuracy Rest: %.4f"%params['train_accuracy_rest'])

    params[f'test_accuracy_{type}'] = np.sum((y_test == 1) & (test_preds == 1)) / np.sum(test_preds == 1)
    params['test_accuracy_rest'] = np.sum((y_test == 0) & (test_preds == 0)) / np.sum(test_preds == 0)

    print(f"Test Accuracy {type}: %.4f"%params[f'test_accuracy_{type}'])
    print(f"Test Accuracy Rest: %.4f"%params['test_accuracy_rest'])

    z_train = np.zeros((2,2))
    z_test = np.zeros((2,2))
    for pred_class in range(2):
        for true_class in range(2):
            z_train[true_class, pred_class] = np.sum((train_preds == pred_class) & (y_train == true_class))
            z_test[true_class, pred_class] = np.sum((test_preds == pred_class) & (y_test == true_class))

In [ ]:
if 'binary' in target_type:
    title_text = 'confusion matrix train'

    x = ['rest', type]
    y = ['rest', type]

    cm_train = px.imshow(z_train, x=x, y=y, color_continuous_scale='Purples', text_auto=True)
    cm_train.update_xaxes(title_text = 'Predicted Label')
    cm_train.update_yaxes(title_text = 'True Label')
    cm_train.update_layout(
        height = 400,
        title_text = title_text,
        title_font_size=20,
        title_x=0.5,
        paper_bgcolor='rgb(229, 237, 247)',
        plot_bgcolor='rgb(229, 237, 247)',    
        )
    cm_train.update_coloraxes(showscale=False)
    cm_train.show()

In [ ]:
if 'binary' in target_type:
    title_text = 'confusion matrix test'

    x = ['rest', type]
    y = ['rest', type]

    cm_test = px.imshow(z_test, x=x, y=y, color_continuous_scale='Purples', text_auto=True)
    cm_test.update_xaxes(title_text = 'Predicted Label')
    cm_test.update_yaxes(title_text = 'True Label')
    cm_test.update_layout(
        height = 400,
        title_text = title_text,
        title_font_size=20,
        title_x=0.5,
        paper_bgcolor='rgb(229, 237, 247)',
        plot_bgcolor='rgb(229, 237, 247)',    
        )
    cm_test.update_coloraxes(showscale=False)
    cm_test.show()

In [ ]:
#--------------regression -------------------
if target_type[:-1] == 'regression':
    params['test_RMSE'] = eval_metric(y_test, test_preds, "RMSE")[0]
    params['train_RMSE'] = eval_metric(y_train, train_preds, "RMSE")[0]
    params['test_R2'] = eval_metric(y_test, test_preds, "R2")[0]
    params['train_R2'] = eval_metric(y_train, train_preds, "R2")[0]
    print("Test  RMSE : %.4f"%params['test_RMSE'])
    print("Train RMSE : % 4f"%params['train_RMSE'])

    print("Test  R2 : %.4f"%params['test_R2'])
    print("Train R2 : % 4f"%params['train_R2'])

In [ ]:
importance_dict = {key:value for value, key in sorted(zip(booster.get_feature_importance(), cols), reverse = False)}
fig1 = px.bar(
    pd.DataFrame(importance_dict.items(), columns = ['features', 'value']),
    x = 'value',
    y = 'features',
    orientation = 'h',
    #title = 'feature importance multiclass',
    title = 'feature importance binary classification away',
    #title = 'feature importance regression team 2',
    width = 600,
    height = 800
)
fig1.show()

In [ ]:
if target_type[:-1] == 'regression':
    fig2 = px.histogram(
        21 * np.array(test_preds, dtype = np.float32)[:1_500_000],
        title = 'score prediction destribution team 2',
        width = 800,
        height = 600
                 )
    fig2.show()

In [24]:
params['description'] = 'Catboost Binary Home'
params['dataset'] = data_version
booster.save_model('./booster.model')

In [25]:
params

{'learning_rate': 0.16656391225752104,
 'l2_leaf_reg': 0.2991233879911666,
 'max_depth': 9,
 'min_data_in_leaf': 15,
 'early_stopping_rounds': 25,
 'verbose': 10,
 'iterations': 200,
 'loss_function': 'RMSE',
 'test_RMSE': 0.05517959516754629,
 'train_RMSE': 0.05447031380292156,
 'test_R2': 0.19840817974788938,
 'train_R2': 0.2120632133214867,
 'description': 'Catboost model team 1',
 'dataset': 'football_live_npz_230105/'}

#### Versioning

In [28]:
target_dict = {'regression1':'FOOT-LIVEBST1',
               'regression2':'FOOT-LIVEBST2',
               'multiclass':'FOOT-LIVEMC', 
               'binary_home':'FOOT-LIVEBC', 
               'binary_draw':'FOOT-LIVEBCDRAW', 
               'binary_away':'FOOT-LIVEBCAWAY'
              }
model_version = neptune.init_model_version(
    model=target_dict[target_type],
    project = 'scomesse/football',
    api_token = api_key # your credentials
)
model_sys = model_version['sys'].fetch()
model_version_params = dict(
    project = 'scomesse/football',
    model = model_sys['model_id'],
    api_token = api_key,
    with_id = model_sys['id']
)
model_version = neptune.init_model_version(**model_version_params)
#model_version['team2_model'].upload('./booster_team2.model')
#model_version['model'].upload('./booster_classification.model')
model_version['model'].upload('./booster.model')
model_version['team_parameters'] = params
model_version.stop()

https://app.neptune.ai/scomesse/football/m/FOOT-LIVEBST1/v/FOOT-LIVEBST1-2
Remember to stop your model_version once you’ve finished logging your metadata (https://docs.neptune.ai/api/model_version#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 15 operations to synchronize with Neptune. Do not kill this process.
All 15 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/scomesse/football/m/FOOT-LIVEBST1/v/FOOT-LIVEBST1-2/metadata


In [29]:
model_version = neptune.init_model_version(**model_version_params)
model_version[f'feature importance'].upload(neptune.types.File.as_html(fig1))
if target_type[:-1] == 'regression':
    model_version[f'score prediction destribution team 2'].upload(neptune.types.File.as_html(fig2))
else:    
    model_version[f'confusion matrix train'].upload(neptune.types.File.as_html(cm_train))
    model_version[f'confusion matrix test'].upload(neptune.types.File.as_html(cm_test))
model_version.stop()

https://app.neptune.ai/scomesse/football/m/FOOT-LIVEBST1/v/FOOT-LIVEBST1-2
Remember to stop your model_version once you’ve finished logging your metadata (https://docs.neptune.ai/api/model_version#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 2 operations to synchronize with Neptune. Do not kill this process.
All 2 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/scomesse/football/m/FOOT-LIVEBST1/v/FOOT-LIVEBST1-2/metadata


In [47]:
poisson.pmf(7, mu = test_preds * 21, loc = 0)

array([3.56332403e-04, 3.04119560e-04, 2.65949632e-04, ...,
       5.89714597e-06, 5.11175603e-06, 4.38099643e-06])